In [ ]:
!pip install transformers datasets torch

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
import torch
from datasets import load_dataset
import evaluate

# Load a smaller dataset (like a subset of IMDb for demonstration)
dataset = load_dataset('imdb', split='train[:5%]')  # Only using 5% of the dataset to reduce memory usage

# Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def preprocess_data(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

# Preprocess the dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Split the dataset for training and evaluation
train_eval_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_eval_split['train']
eval_dataset = train_eval_split['test']

# Load the DistilBERT model (pre-trained)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Metric for evaluation (accuracy)
accuracy_metric = evaluate.load("accuracy")

# Function to compute accuracy
def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# 1. EVALUATE BEFORE FINE-TUNING (using the pre-trained model)
print("Evaluating before fine-tuning...")

# Set up the trainer (no training yet, only evaluation)
trainer_before = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./results',
        per_device_eval_batch_size=8,   # Use smaller eval batch size
    ),
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_accuracy
)

# Evaluate the model before fine-tuning
eval_results_before = trainer_before.evaluate()
print(f"Accuracy before fine-tuning: {eval_results_before['eval_accuracy']}")

# 2. FINE-TUNE THE MODEL
print("Fine-tuning the model...")

# Set the training arguments for fine-tuning
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    num_train_epochs=1,  # Fewer epochs to save time
    per_device_train_batch_size=8,  # Smaller batch size to fit in memory
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    eval_accumulation_steps=2,   # To reduce memory consumption
    fp16=True if torch.cuda.is_available() else False,  # Mixed precision for GPUs
)

# Trainer for fine-tuning
trainer_finetune = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_accuracy
)

# Fine-tune the model
trainer_finetune.train()

# 3. EVALUATE AFTER FINE-TUNING
print("Evaluating after fine-tuning...")

# Evaluate the model after fine-tuning
eval_results_after = trainer_finetune.evaluate()
print(f"Accuracy after fine-tuning: {eval_results_after['eval_accuracy']}")

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_distilbert")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000600,0.000458,1.000000


Evaluating after fine-tuning...


Accuracy after fine-tuning: 1.0
